In [ ]:
# load libraries and scripts
using Statistics
using Pkg
using HDF5
using SparseArrays
using LinearAlgebra
using Plots
using Flux
using Random
using DelimitedFiles
using LaTeXStrings
Pkg.activate("..")
using ContGridMod

In [ ]:
# Constants
NTRAIN = 48;
NTEST = 12;

In [ ]:
# load border and create the mesh
border, scaleFactor = import_border("../data/borders/euro_border.json")
dx = 0.011
mesh = get_mesh(border, dx);

In [ ]:
# load all discrete models
trainingDiscMod = ContGridMod.DiscModel[]
testDiscMod = ContGridMod.DiscModel[]
for i=1:NTRAIN
    push!(trainingDiscMod, load_discrete_model("../data/ml/training_" * string(i) * ".h5", scaleFactor))
end
for i=1:NTEST
    push!(testDiscMod, load_discrete_model("../data/ml/test_" * string(i) * ".h5", scaleFactor))
end

In [ ]:
# Create the continuous model and obtain the adjacency list
contmod = get_params(mesh, scaleFactor, "../data/disc/pantagruel.h5", Niter=10,
    dmax = 2*dx, patch=100.0, tau=5.0e-6);
adjList = contmod.mesh.inc_mat;

In [ ]:
trainingTheta = zeros(size(trainingDiscMod[1].th, 1), NTRAIN)
testTheta = zeros(size(testDiscMod[1].th, 1), NTEST)
trainingP = zeros(size(contmod.p, 1), NTRAIN)
testP = zeros(size(contmod.p, 1), NTEST)
for i=1:NTRAIN
    update_model!(contmod, trainingDiscMod[i])
    trainingP[:, i] = contmod.p
    trainingTheta[:, i] = trainingDiscMod[i].th
end
for i=1:NTEST
    update_model!(contmod, testDiscMod[i])
    testP[:, i] = contmod.p
    testTheta[:, i] = testDiscMod[i].th
end

In [ ]:
# Find the ids of the nodes that map to the discrete nodes. 
ids = ContGridMod.get_discrete_id(contmod.mesh.coord, trainingDiscMod[1].coord)
# Map quantities from the continuous model onto the discrete one
contToDisc = sparse(1:length(ids), ids, ones(length(ids)), length(ids), size(contmod.mesh.coord, 1));

In [ ]:
# Find slack bus and remove from ids, create map from grounded to full model
nNodes = size(contmod.xi, 1);
idSlack = find_node(contmod, reshape(trainingDiscMod[1].coord[trainingDiscMod[1].idgen[1], :], 1, 2))[1];
idsWoSlack = setdiff(1:nNodes, idSlack);
unground = sparse(idsWoSlack, 1:nNodes-1, ones(nNodes-1), nNodes, nNodes-1);

In [ ]:
# Create the grounded incidence matrices and ground the power matrices
nEdges = size(adjList, 1);
incMat = sparse([adjList[:,1]; adjList[:,2]], [1:nEdges; 1:nEdges], [-ones(nEdges); ones(nEdges)]);
incMat = incMat[idsWoSlack,:];
incMatTrans = SparseMatrixCSC{Float64, Int64}(incMat');
trainingPGround = trainingP[idsWoSlack, :];
testPGround = testP[idsWoSlack, :];

In [ ]:
# Initialize machine learning
opt = ADAM(0.1);
b = 100 * ones(nEdges);
param = Flux.params(b);

In [ ]:
nEpochs = 10000;
nBatches = 3;
batchSize = Int64(NTRAIN / nBatches);
shuffledIx = randperm(NTRAIN);

In [ ]:
@time begin
err = zeros(nEpochs * nBatches)
for e=1:nEpochs
    for batch=1:nBatches
        local _err
        gs = gradient(param) do
            bTemp = max.(b, 0.1)
            lap = incMat * (bTemp .* incMatTrans)
            theta = unground * (lap \ trainingPGround[:,shuffledIx[(batch - 1) * batchSize + 1:batch * batchSize]] * contmod.mesh.dx^2)
            _err = mean(abs2, trainingTheta[:,shuffledIx[(batch - 1) * batchSize + 1:batch * batchSize]] - contToDisc * theta)
            return _err
        end
        if(mod(e,50) == 0 && batch == 1)
            println([e _err])
        end
        err[(e - 1) * nBatches + batch] = _err
        # push!(err, _err)
        Flux.update!(opt, param, gs)
    end
end
end
b = max.(b, 0.1);

In [ ]:
h5write("../data/ml/susceptances.h5", "b", b);

In [ ]:
b = h5read("../data/ml/susceptances.h5", "b");

In [ ]:
# Calculate Errors
lap = incMat * (b .* incMatTrans)
trainingPredCont = unground * (lap \ trainingPGround * contmod.mesh.dx^2);
testPredCont = unground * (lap \ testPGround * contmod.mesh.dx^2);
trainingPred = contToDisc * unground * (lap \ trainingPGround * contmod.mesh.dx^2);
testPred = contToDisc * unground * (lap \ testPGround * contmod.mesh.dx^2);
trainingError = mean(abs2, trainingTheta - trainingPred);
testError = mean(abs2, testTheta - testPred);
println("Training Error: ", trainingError)
println("Test Error: ", testError)

In [ ]:
plot(LinRange(1, nEpochs, nEpochs * nBatches), err, yaxis=:log10, label=:none, xlabel="Epoch", ylabel="Training Error")

In [ ]:
trainingPlots = Plots.Plot[]

for i=1:NTRAIN
    min, max = extrema([trainingTheta[:,i] trainingPred[:,i]])
    delta = max - min
    pad = 0.05 * delta
    min -= pad
    max += pad
push!(trainingPlots, scatter(trainingTheta[:,i], trainingPred[:,i], xlims=(min, max), ylims=(min, max), label="Training Set " * string(i)))
end

plot(trainingPlots..., layout=(12,4), size=(1500, 3000))


In [ ]:
testPlots = Plots.Plot[]

for i=1:NTEST
    min, max = extrema([testTheta[:,i] testPred[:,i]])
    delta = max - min
    pad = 0.05 * delta
    min -= pad
    max += pad
push!(testPlots, scatter(testTheta[:,i], testPred[:,i], xlims=(min, max), ylims=(min, max), label="Test Set " * string(i)))
end

plot(testPlots..., layout=(4,3), size=(1125, 1000))

In [ ]:
writedlm("../plotutil/trainingactual.csv", trainingTheta, ',')
writedlm("../plotutil/testactual.csv", testTheta, ',')
writedlm("../plotutil/trainingprediction.csv", trainingPred, ',')
writedlm("../plotutil/testPrediction.csv", testPred, ',')

In [ ]:
# Load the country in which each bus is in
countries = readdlm("../data/ml/countries.csv", ',');

In [ ]:
# Load the German border
borderDe = albers_projection(readdlm("../data/borders/germany.csv", ',')/180*π)/scaleFactor;
inDe = ContGridMod.inPolygon(contmod.mesh.coord, borderDe);

In [ ]:
# Calculate the power flows into Germany in the discrete model
linesDeFirstDisc = Int64[]
linesDeSecondDisc = Int64[]
for (i, l) in enumerate(eachrow(trainingDiscMod[1].idb))
    if countries[l[1]] == "DE" && countries[l[2]] != "DE"
        push!(linesDeFirstDisc, i)
    elseif countries[l[2]] == "DE" && countries[l[1]] != "DE"
        push!(linesDeSecondDisc, i)
    end
end
trainingPowerflowDeDisc = zeros(NTRAIN)
testPowerflowDeDisc = zeros(NTEST)
for l in linesDeFirstDisc
    trainingPowerflowDeDisc .-= trainingDiscMod[1].bline[l] * sin.(trainingTheta[trainingDiscMod[1].idb[l, 1], :]-trainingTheta[trainingDiscMod[1].idb[l, 2], :])
    testPowerflowDeDisc .-= testDiscMod[1].bline[l] * sin.(testTheta[testDiscMod[1].idb[l, 1], :]-testTheta[testDiscMod[1].idb[l, 2], :])
end
for l in linesDeSecondDisc
    trainingPowerflowDeDisc .+= trainingDiscMod[1].bline[l] * sin.(trainingTheta[trainingDiscMod[1].idb[l, 1], :]-trainingTheta[trainingDiscMod[1].idb[l, 2], :])
    testPowerflowDeDisc .+= testDiscMod[1].bline[l] * sin.(testTheta[testDiscMod[1].idb[l, 1], :]-testTheta[testDiscMod[1].idb[l, 2], :])
end

In [ ]:
# Calculate the power flows into Germany in the continuous model
linesDeFirstCont = Int64[]
linesDeSecondCont = Int64[]
for (i, l) in enumerate(eachrow(adjList))
    if inDe[l[1]] && !inDe[l[2]]
        push!(linesDeFirstCont, i)
    elseif inDe[l[2]] && !inDe[l[1]]
        push!(linesDeSecondCont, i)
    end
end
trainingPowerflowDeCont = zeros(NTRAIN)
testPowerflowDeCont = zeros(NTEST)
for l in linesDeFirstCont
    trainingPowerflowDeCont .-= b[l] * sin.(trainingPredCont[adjList[l, 1], :]-trainingPredCont[adjList[l, 2], :])
    testPowerflowDeCont .-= b[l] * sin.(testPredCont[adjList[l, 1], :]-testPredCont[adjList[l, 2], :])
end
for l in linesDeSecondCont
    trainingPowerflowDeCont .+= b[l] * sin.(trainingPredCont[adjList[l, 1], :]-trainingPredCont[adjList[l, 2], :])
    testPowerflowDeCont .+= b[l] * sin.(testPredCont[adjList[l, 1], :]-testPredCont[adjList[l, 2], :])
end

In [ ]:
plot(trainingPowerflowDeDisc/10, label="Discrete Model", title="Flows into Germany, Training Set", xlabel="Set", ylabel=L"P\,[\mathrm{GW}]")
plot!(trainingPowerflowDeCont/10, label="Continuous Model")

In [ ]:
plot(testPowerflowDeDisc/10, label="Discrete Model", title="Flows into Germany, Test Set", xlabel="Set", ylabel=L"P\,[\mathrm{GW}]",xticks=[0,2,4,6,8,10,12])
plot!(testPowerflowDeCont/10, label="Continuous Model")

In [ ]:
# Load the Italian border
borderIt = albers_projection(readdlm("../data/borders/italy.csv", ',')/180*π)/scaleFactor;
inIt = ContGridMod.inPolygon(contmod.mesh.coord, borderIt);

In [ ]:
# Calculate the power flows into Germany in the discrete model
linesItFirstDisc = Int64[]
linesItSecondDisc = Int64[]
for (i, l) in enumerate(eachrow(trainingDiscMod[1].idb))
    if countries[l[1]] == "IT" && countries[l[2]] != "IT"
        push!(linesItFirstDisc, i)
    elseif countries[l[2]] == "IT" && countries[l[1]] != "IT"
        push!(linesItSecondDisc, i)
    end
end
trainingPowerflowItDisc = zeros(NTRAIN)
testPowerflowItDisc = zeros(NTEST)
for l in linesItFirstDisc
    trainingPowerflowItDisc .-= trainingDiscMod[1].bline[l] * sin.(trainingTheta[trainingDiscMod[1].idb[l, 1], :]-trainingTheta[trainingDiscMod[1].idb[l, 2], :])
    testPowerflowItDisc .-= testDiscMod[1].bline[l] * sin.(testTheta[testDiscMod[1].idb[l, 1], :]-testTheta[testDiscMod[1].idb[l, 2], :])
end
for l in linesItSecondDisc
    trainingPowerflowItDisc .+= trainingDiscMod[1].bline[l] * sin.(trainingTheta[trainingDiscMod[1].idb[l, 1], :]-trainingTheta[trainingDiscMod[1].idb[l, 2], :])
    testPowerflowItDisc .+= testDiscMod[1].bline[l] * sin.(testTheta[testDiscMod[1].idb[l, 1], :]-testTheta[testDiscMod[1].idb[l, 2], :])
end

In [ ]:
# Calculate the power flows into Germany in the continuous model
linesItFirstCont = Int64[]
linesItSecondCont = Int64[]
for (i, l) in enumerate(eachrow(adjList))
    if inIt[l[1]] && !inIt[l[2]]
        push!(linesItFirstCont, i)
    elseif inIt[l[2]] && !inIt[l[1]]
        push!(linesItSecondCont, i)
    end
end
trainingPowerflowItCont = zeros(NTRAIN)
testPowerflowItCont = zeros(NTEST)
for l in linesItFirstCont
    trainingPowerflowItCont .-= b[l] * sin.(trainingPredCont[adjList[l, 1], :]-trainingPredCont[adjList[l, 2], :])
    testPowerflowItCont .-= b[l] * sin.(testPredCont[adjList[l, 1], :]-testPredCont[adjList[l, 2], :])
end
for l in linesItSecondCont
    trainingPowerflowItCont .+= b[l] * sin.(trainingPredCont[adjList[l, 1], :]-trainingPredCont[adjList[l, 2], :])
    testPowerflowItCont .+= b[l] * sin.(testPredCont[adjList[l, 1], :]-testPredCont[adjList[l, 2], :])
end

In [ ]:
plot(trainingPowerflowItDisc/10, label="Discrete Model", title="Flows into Italy, Training Set", xlabel="Set", ylabel=L"P\,[\mathrm{GW}]")
plot!(trainingPowerflowItCont/10, label="Continuous Model")

In [ ]:
plot(testPowerflowItDisc/10, label="Discrete Model", title="Flows into Italy, Test Set", xlabel="Set", ylabel=L"P\,[\mathrm{GW}]", xticks=[0,2,4,6,8,10,12])
plot!(testPowerflowItCont/10, label="Continuous Model")

In [ ]:
writedlm("../plotutil/powerflowsde.csv", [trainingPowerflowDeDisc trainingPowerflowDeCont; testPowerflowDeDisc testPowerflowDeCont], ',')
writedlm("../plotutil/powerflowsit.csv", [trainingPowerflowItDisc trainingPowerflowItCont; testPowerflowItDisc testPowerflowItCont], ',')